### setup

In [1]:
%load_ext lab_black

In [2]:
#%pip install cartopy
#%pip install vega_datasets

In [3]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import geopandas as gpd
import us
from cartopy import crs as ccrs
from vega_datasets import data

In [38]:
import matplotlib

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [6]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

### read data

In [7]:
# 1980-2022 Billion-Dollar Disasters Cost (in $Millions) By State (CPI-Adjusted)
state_df = pd.read_csv("data/raw/hurricanes/state-cost-data.csv", skiprows=1)

In [8]:
# Billion-Dollar Disasters By Year (CPI-Adjusted)
over_time = pd.read_csv("data/raw/hurricanes/state-freq-data.csv", skiprows=1).rename(
    columns={"state": "abbr"}
)

In [9]:
all_features = []

for st in us.states.STATES:
    states_dict = {
        "fips": st.fips,
        "name": st.name,
        "abbr": st.abbr,
    }
    all_features.append(states_dict)

In [10]:
postal_name = pd.DataFrame(all_features)

In [11]:
df = over_time.merge(postal_name, on="abbr")

In [12]:
df.columns = df.columns.str.replace(" ", "_")

In [13]:
df.head()

,year,abbr,drought,flooding,freeze,severe_storm,tropical_cyclone,wildfire,winter_storm,fips,name
0,1980,AK,0,0,0,0,0,0,0,02,Alaska
1,1981,AK,0,0,0,0,0,0,0,02,Alaska
2,1982,AK,0,0,0,0,0,0,0,02,Alaska
3,1983,AK,0,0,0,0,0,0,0,02,Alaska
4,1984,AK,0,0,0,0,0,0,0,02,Alaska


In [27]:
df_slim = df[df["year"] >= 2017][
    ["year", "abbr", "name", "fips", "tropical_cyclone"]
].fillna(0)

In [28]:
states = gpd.read_file("data/raw/hurricanes/usa_counties_esri_simple_mainland.geojson")

In [29]:
albo = ccrs.AlbersEqualArea(
    central_longitude=-96,
    central_latitude=37.5,
    false_easting=0.0,
    false_northing=0.0,
    standard_parallels=(29.5, 45.5),
    globe=None,
)

In [30]:
gdf = gpd.GeoDataFrame(df_slim.merge(states, on=["name", "fips"]))

In [31]:
gdf = gdf.to_crs(albo.proj4_init)
states = states.to_crs(albo.proj4_init)

In [32]:
years = gdf.year.sort_values().unique()

In [35]:
alt.Chart(gdf).mark_geoshape().encode(
    shape="geometry:G",
    color="severe_storm",
    facet=alt.Facet("year", columns=2),
).properties(width=300, height=175,).project(type="albersUsa")

alt.Chart(...)

In [ ]:
for y in years:
    ax = gdf[gdf["year"] == y].boundary.plot(
        edgecolor="white", linewidth=0.2, figsize=(10, 7)
    )
    geometry.plot(ax=ax, facecolor="#e9e9e9", edgecolor="#fff", linewidth=0.4)
    gdf[gdf["year"] == y].plot(
        ax=ax,
        column="severe_storm",
        legend=False,
        categorical=True,
        scheme="User_Defined",
        classification_kwds=dict(bins=breaks_manual),
    )